# Grid Search for Optimal Hyperparameter Values

To find the optimal hyperparameter values for the *learning rate*, *minimum amount of data in each leaf*, and *maximum number of leaves on each node*, we'll look through the RMSE values for each of the experiments (averaged of 10 replications for each configuration). We will do this for four types of model: MVN NGBoost trained with and without SST gradients and MVN NGBoost trained with polar velocities or cartesian velocities.

The hyperparameter options for the grid search are:

* Learning rate, $\eta$: $\{0.01, 0.1, 1.0\}$
* Minimum data per leaf: $\{1, 15, 32\}$
* Maximum leaves per branch: $\{8, 15, 32, 64\}$

Along with the fixed model hyperparameters:

* Maximum boosting iterations, $B = 1000$
* Early stopping rounds: $50$
* Maximum tree depth: $15$


In [1]:
# load modules and the RMSE values for each configuration

import pickle
import os
import sys
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('hpc_scripts'), '..')))
from experiments.hpc_scripts.config_search import model_config

with open(r'experiment_date_04-02-2025_grid_search.p', 'rb') as pickle_file:
    grid = pickle.load(pickle_file)

# model configurations path
model_filepath = r'../model_configuration_ids.p'

In [2]:
grid #figure out a way to only include ones that have all 10 replications present

,RMSE
config ID,
39,14.551255
38,14.584261
7,14.586115
43,14.590803
35,14.610322
...,...
90,201.289281
56,481.087466
58,506.559118


In [3]:
# add sst_flag and polar_flag to df

vals = grid.index.map(lambda id: model_config(id,model_filepath=model_filepath)[3:5]).values
grid[['sst_flag','polar_flag']] = np.array([val for val in vals])

In [4]:
print('\nWorst performing hyperparameter configurations')
ids = grid.tail(5).index.values

for id in ids:
    model_config(id,model_filepath,verbose=True)
    print('\n RMSE \n--------------------------------')
    print(f'{grid.loc[id,'RMSE']:.2f} cm/s')


Worst performing hyperparameter configurations

config ID: 90

 Hyperparameter Values
---------------------------
learning rate: 1
min. data per leaf: 32
max. leaves per node: 32

 Data Information
---------------------------
Uses SST gradient: True
Uses velocities in polar form: False

 RMSE 
--------------------------------
201.29 cm/s

config ID: 56

 Hyperparameter Values
---------------------------
learning rate: 1
min. data per leaf: 16
max. leaves per node: 32

 Data Information
---------------------------
Uses SST gradient: False
Uses velocities in polar form: False

 RMSE 
--------------------------------
481.09 cm/s

config ID: 58

 Hyperparameter Values
---------------------------
learning rate: 1
min. data per leaf: 16
max. leaves per node: 32

 Data Information
---------------------------
Uses SST gradient: True
Uses velocities in polar form: False

 RMSE 
--------------------------------
506.56 cm/s

config ID: 51

 Hyperparameter Values
---------------------------
learn

In [5]:
# identify the top parameters for each combination of (SST_flag, polar_flag)
# sst flag: False, polar_flag: False
grid_sst_false_polar_false = grid.query('not sst_flag and not polar_flag')[['RMSE']]
# sst flag: False, polar_flag: True
grid_sst_false_polar_true = grid.query('not sst_flag and polar_flag')[['RMSE']]
# sst flag: True, polar_flag: False
grid_sst_true_polar_false = grid.query('sst_flag and not polar_flag')[['RMSE']]
# sst flag: True, polar_flag: True
grid_sst_true_polar_true = grid.query('sst_flag and polar_flag')[['RMSE']]


In [6]:
grid_sst_false_polar_false

,RMSE
config ID,
36,14.80333
40,14.822587
0,14.840364
4,14.841547
8,14.853947
32,14.855648
44,14.874416
12,14.902195
20,14.934338


In [7]:
print('\nOptimal Hyperparameter Configurations')

print(f'\nFor sst_flag: False, polar_flag: False \n Config ID: {grid_sst_false_polar_false.head(1).index.values[0]}')
print(f'\nFor sst_flag: False, polar_flag: True \n Config ID: {grid_sst_false_polar_true.head(1).index.values[0]}')
print(f'\nFor sst_flag: True, polar_flag: False \n Config ID: {grid_sst_true_polar_false.head(1).index.values[0]}')
print(f'\nFor sst_flag: True, polar_flag: True \n Config ID: {grid_sst_true_polar_true.head(1).index.values[0]}')


Optimal Hyperparameter Configurations

For sst_flag: False, polar_flag: False 
 Config ID: 36

For sst_flag: False, polar_flag: True 
 Config ID: 37

For sst_flag: True, polar_flag: False 
 Config ID: 38

For sst_flag: True, polar_flag: True 
 Config ID: 39


In [8]:
for id in [36,37,38,39]:
    model_config(id, model_filepath,verbose=True)

    print('\n RMSE \n --------------------------------')
    print(f'{grid.loc[id,'RMSE']:.2f} cm/s')

with open(model_filepath, 'rb') as filehandler:
    configs = pickle.load(filehandler)


config ID: 36

 Hyperparameter Values
---------------------------
learning rate: 0.1
min. data per leaf: 32
max. leaves per node: 16

 Data Information
---------------------------
Uses SST gradient: False
Uses velocities in polar form: False

 RMSE 
 --------------------------------
14.80 cm/s

config ID: 37

 Hyperparameter Values
---------------------------
learning rate: 0.1
min. data per leaf: 32
max. leaves per node: 16

 Data Information
---------------------------
Uses SST gradient: False
Uses velocities in polar form: True

 RMSE 
 --------------------------------
14.79 cm/s

config ID: 38

 Hyperparameter Values
---------------------------
learning rate: 0.1
min. data per leaf: 32
max. leaves per node: 16

 Data Information
---------------------------
Uses SST gradient: True
Uses velocities in polar form: False

 RMSE 
 --------------------------------
14.58 cm/s

config ID: 39

 Hyperparameter Values
---------------------------
learning rate: 0.1
min. data per leaf: 32
max. 

In [32]:
config_id = int(np.floor((1440-1)/10))

In [33]:
config_id

143